# Assignment 2 - Part 3

```
Team 37
Avani Gupta
Bhuvanesh Sridharan
```

In [2]:
!python3 -m pip install cvxpy

Defaulting to user installation because normal site-packages is not writeable
  Using cached cvxpy-1.0.31.tar.gz (947 kB)
  Installing build dependencies ... error
  ERROR: Command errored out with exit status 1:
   command: /usr/bin/python3 /home/animesh/.local/lib/python3.6/site-packages/pip install --ignore-installed --no-user --prefix /tmp/pip-build-env-aw2kqp4m/overlay --no-warn-script-location --no-binary :none: --only-binary :none: -i https://pypi.org/simple -- 'numpy >= 1.15' 'scipy >= 1.1.0' 'setuptools>=40.8.0' wheel
       cwd: None
  Complete output (39 lines):
  Traceback (most recent call last):
    File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "/home/animesh/.local/lib/python3.6/site-packages/pip/__main__.py", line 16, in <module>
      from pip._internal.cli.main import main as _main  # isort:skip # noqa
    File "/home/

In [1]:
import numpy as np
import cvxpy as cp

ModuleNotFoundError: No module named 'cvxpy'

State is of the form $\longrightarrow [\text{enemy_health}\in\{0..4\}, \text{arrows}\in\{0..3\},\text{stamina}\in\{0..2\}]$

Action $\in$ {NOOP, SHOOT, DODGE, RECHARGE}

In [ ]:
ACTIONS = ["NOOP", "SHOOT", "DODGE", "RECHARGE" ]

In [2]:
def kronecker(i :int, j :int):
    if i==j:
        return 1
    else:
        return 0

In [4]:
def intToList(s :int):
    return [s//12, (s%12)//3, s%3]

def listToInt(s :list):
    return s[0]*12 + s[1]*3 + s[2]

In [5]:
def gameLogic(s:int, a:int):
    """
    Params: s=State, a=action
    Returns a dictionary with 
    key: state
    value: probability
    when action a is performed in state s"""
    probs = dict()
    if ACTIONS[a] == "NOOP":
        # enemy health = 0 then NOOP gives back same state with
        # prob = 1.0
        if intToList(s)[0] == 0:
            probs[s] = 1.0
        return probs
    
    # If action is not NOOP and MD health is 0 then no action can be taken
    if intToList(s)[0] == 0:
        return probs
    
    if ACTIONS[a] == "SHOOT":     
        # Checking that stamina and arrow not zero:
        if intToList(s)[1] != 0 and intToList(s)[2] != 0:
            temp = intToList(s)
            
            # decreasing stamina and arrows by 1
            temp[1] -= 1
            temp[2] -= 1
            
            # 0.5 prob that health stays same:
            probs[listToInt(temp)] = 0.5
            
            # prob that the health decreases.
            temp[0] -= 1
            probs[listToInt(temp)] = 0.5
            
    if ACTIONS[a] == "DODGE":
        state = intToList(s)
        if state[2] == 0: # stamina 0
            return probs
        
        if state[2] == 1: #stamina 50
            
            if state[1] != 3: #arrows not full
                
                # arrow picked
                temp = state.copy()
                temp[2] = 0
                temp[1] += 1
                probs[listToInt(temp)] = 0.8
                
                # arrow not picked
                temp = state.copy()
                temp[2] = 0
                probs[listToInt(temp)] = 0.2
                
                return probs
            
            if state[1] == 3: #Arrows Full
                temp = state.copy()
                temp[2] = 0
                probs[listToInt(temp)] = 1.0
                return probs
            
        if state[2] == 2: #stamina is 100
            
            if state[1] != 3: #arrows not full
                
                # stamina decreases to 50 and arrow picked
                temp = state.copy()
                temp[2] = 1
                temp[1] += 1
                probs[listToInt(temp)] = 0.8 * 0.8
                
                # stamina decreases to 0 and arrow picked
                temp = state.copy()
                temp[2] = 0
                temp[1] += 1
                probs[listToInt(temp)] = 0.2 * 0.8
                
                #stamina decreases to 50 and arrow not picked
                temp = state.copy()
                temp[2] = 1
                probs[listToInt(temp)] = 0.8 * 0.2
                
                #stamina decreased to 0 and arrow not picked
                temp = state.copy()
                temp[2] = 0
                probs[listToInt(temp)] = 0.2 * 0.2
                
                return probs
            
            if state[1] == 3: #arrows full
                
                # stamina decreases to 50:
                temp = state.copy()
                temp[2] = 1
                probs[listToInt(temp)] = 0.8
                
                #stamina decreases to 0:
                temp = state.copy()
                temp[2] = 0
                probs[listToInt(temp)] = 0.2
                
                return probs
    if ACTIONS[a] == "RECHARGE":
        state = intToList(s)
        
        if state[2] == 2: #stamina is 100
            return probs
        
        if state[2] != 2:
            
            # stamina increases by 50
            temp = state.copy()
            temp[2] += 1
            probs[listToInt(temp)] = 0.8
            
            #stamina stays the same
            temp = state.copy()
            probs[listToInt(temp)] = 0.2
            
            return probs
    return probs
        
    

In [6]:
def P(i :int , j:int , a:int):
    """
    returns probabilty of system going to state j
    when action a is performed in state i
    """
    
    probs = gameLogic(i, a)
    
    return probs.get(j, 0.0)

## Creating array A:

x = Array with shape  $(4 \times 60, 1)$

$\alpha$ = Array with shape $(60, 1)$

A = Array with shape $(60, 4 \times 60)$ 

X = $[ x_{00}, x_{01}, x_{02}, ... ]$

In [7]:
A = np.zeros((60,4*60))

In [8]:
for x_ind in range(4*60):
    for j in range(60):
        A[j][x_ind] = kronecker(x_ind//4, j) - P(x_ind//4, j, x_ind%4)

### Creating $\alpha$ 

All elements 0 except begin state

In [9]:
alpha = np.zeros((60,1))
start_state = [4,3,2]
alpha[listToInt(start_state)][0] = 1

In [10]:
listToInt(start_state)

59

### Creating R (Reward funcition for action at each state)

Dimentions : $(1, 4 \times 60)$



In [11]:
teamNumber = 37
penaltyArray = [0.5, 1, 2]
penalty = -10/penaltyArray[teamNumber%3]
print("Penalty for action : "+ str(penalty))

Penalty for action : -10.0


In [12]:
R = np.zeros((1,4*60))

for x_ind in range(4*60):
    
    R[0][x_ind] = penalty
    state = x_ind//4
    action = x_ind%4
    ninf = -100 # using as negative infinity
    stateList = intToList(state)
    
    if stateList[0] == 0: #dragon health 0
        if ACTIONS[action] == "NOOP":
            R[0][x_ind] = 0
        else :
            R[0][x_ind] = ninf
            A[:,x_ind] = ninf
        continue
            
    if stateList[2] == 0: #stamina is 0
        if ACTIONS[action] == "SHOOT" or ACTIONS[action] == "DODGE":
            R[0][x_ind] = ninf
            A[:,x_ind] = ninf
                
    if stateList[2] == 2: #stamina is 100
        if ACTIONS[action] == "RECHARGE":
            R[0][x_ind] = ninf
            A[:,x_ind] = ninf
        
    if stateList[1] == 0: #arrows is 0
        if ACTIONS[action] == "SHOOT":
            R[0][x_ind] = ninf
            A[:,x_ind] = ninf


In [15]:
A

array([[   0. , -100. , -100. , ...,    0. ,    0. , -100. ],
       [   0. , -100. , -100. , ...,    0. ,    0. , -100. ],
       [   0. , -100. , -100. , ...,    0. ,    0. , -100. ],
       ...,
       [   0. , -100. , -100. , ...,    0. ,   -0.2, -100. ],
       [   0. , -100. , -100. , ...,    0. ,   -0.8, -100. ],
       [   0. , -100. , -100. , ...,    1. ,    1. , -100. ]])

In [16]:
x = cp.Variable(shape=(240,1), name="x")

constraints = [cp.matmul(A, x) <= alpha,cp.matmul(A, x) >= alpha, x>=0]
objective = cp.Maximize(cp.matmul(R,x))
problem = cp.Problem(objective, constraints)

solution = problem.solve()
print(solution)

print(x.value[listToInt([1,3,2])*4:])
np.matmul(R,x.value)

-9.999999999974833
[[ 2.71500348e-14]
 [ 5.43693929e-14]
 [ 5.81678658e-14]
 [-2.58021052e-16]
 [ 3.50049145e-14]
 [-2.58021012e-16]
 [-2.58021012e-16]
 [ 6.40666442e-14]
 [ 3.37310664e-14]
 [-2.58021011e-16]
 [ 5.79316263e-14]
 [ 6.43645201e-14]
 [ 3.20653653e-14]
 [-2.58021012e-16]
 [ 6.14289955e-14]
 [-2.58021052e-16]
 [ 3.99017864e-14]
 [-2.58021011e-16]
 [-2.58021011e-16]
 [ 6.91017045e-14]
 [ 3.21385944e-14]
 [ 6.03093357e-14]
 [ 5.60918415e-14]
 [ 6.68844085e-14]
 [ 2.71432170e-14]
 [ 5.67075469e-14]
 [ 5.70513298e-14]
 [-2.58021012e-16]
 [ 3.90379325e-14]
 [-2.58021052e-16]
 [-2.58021012e-16]
 [ 6.86519336e-14]
 [ 3.18245327e-14]
 [ 5.54316498e-14]
 [ 5.60834233e-14]
 [ 6.66840250e-14]
 [ 2.70678984e-14]
 [ 5.81732383e-14]
 [ 5.78954519e-14]
 [-2.58021052e-16]
 [ 3.88664354e-14]
 [-2.58021012e-16]
 [-2.58021011e-16]
 [ 6.94728462e-14]
 [ 3.05294789e-14]
 [ 5.46757541e-14]
 [ 5.48526927e-14]
 [ 6.57460661e-14]
 [ 2.70325677e-14]
 [ 5.83131006e-14]
 [ 5.80677626e-14]
 [-2.5802101

array([[-10.]])